# **Automatic Translation**

* Dado que **regular tensors**  tienen formas
fijas, solo pueden contener oraciones de la misma longitud. Puede usar **masking** para manejar esto (es decir que ignore los tokens de relleno-pad). Sin
embargo, si las oraciones tienen longitudes muy diferentes, no puede
simplemente recortarlas como hicimos para sentiment analysis.   En su lugar, agrupe las oraciones en buckets(cubos) de longitud similar(e.g., a bucket for the 1- to 6-word
sentences, another for the 7- to 12-word sentences, and so on),
usando pad(relleno) para las secuencias más cortas para asegurarse de que todas las
oraciones en un bucket tienen la misma longitud
(consulte la función `tf.data.experimental.bucket_by_sequence_length()` para
esto).

* Queremos ignorar cualquier salida más allá del token EOS, por lo que estos
tokens no deberían contribuir a la pérdida (deben enmascararse)

* Cuando el vocabulario de salida es grande , generar
una probabilidad para cada una de las palabras  sería
terriblemente lento.  Para evitar esto, una solución es mirar solo los logits generados por 
el modelo para encontrar la palabra correcta y una muestra aleatoria de palabras incorrectas, luego
calcular una aproximación de la pérdida basada solo en estos logits. Esta tecnica es llamada` sampled softmax` y en tensorflow  se puede usar `tf.nn.sampled_softmax_loss()` durante el entrenamiento pero la funcion sofmax normal durante la inferencia (sampled softmax cannot be used at inference time because it
requires knowing the target)


El proyecto `TensorFlow Addons` incluye muchas herramientas de *sequence-to-sequence** que le permiten crear fácilmente **encoder-decoder** listos
para la producción.  TFA ya no se desarrollará activamente, seguirá recibiendo soporte mínimo para corrección de errores y actualizaciones de seguridad hasta su fin de vida en mayo de 2024.

In [5]:
import tensorflow_addons as tfa
from tensorflow import keras
import numpy as np
import tensorflow as tf
tf.random.set_seed(42)

vocab_size = 100
embed_size = 10

encoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs = keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths = keras.layers.Input(shape=[], dtype=np.int32)

embeddings = keras.layers.Embedding(vocab_size, embed_size)

encoder_embeddings = embeddings(encoder_inputs)
decoder_embeddings = embeddings(decoder_inputs)

# return_state=True para que podamos obtener su estado oculto final y pasarlo al decoder
encoder = keras.layers.LSTM(512, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_embeddings)

# returns two hidden states (short term and long term)
encoder_state = [state_h, state_c]


# The TrainingSampler role is to tell the decoder at each step what it
# should pretend the previous output was
sampler = tfa.seq2seq.sampler.TrainingSampler()

#  basic LSTMcell
decoder_cell = keras.layers.LSTMCell(512)

output_layer = keras.layers.Dense(vocab_size)
decoder = tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler,
                                                 output_layer=output_layer)

final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, initial_state=encoder_state,
    sequence_length=sequence_lengths)
Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])


In [6]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")


X = np.random.randint(100, size=10*1000).reshape(1000, 10)
Y = np.random.randint(100, size=15*1000).reshape(1000, 15)

X_decoder = np.c_[np.zeros((1000, 1)), Y[:, :-1]]
seq_lengths = np.full([1000], 15)

history = model.fit([X, X_decoder, seq_lengths], Y, epochs=2)



Epoch 1/2
32/32 [==============================] - 23s 202ms/step - loss: 4.6056
Epoch 2/2
32/32 [==============================] - 7s 220ms/step - loss: 4.6035


### Bidirectional Recurrent Layers

The Bidirectional layer will create a clone of the GRU layer (but in the reverse
direction), and it will run both and concatenate their outputs. So although the GRU
layer has 10 units, the Bidirectional layer will output 20 values per time step.

In [7]:
model = keras.models.Sequential([
    keras.layers.GRU(10, return_sequences=True, input_shape=[None, 10]),
    keras.layers.Bidirectional(keras.layers.GRU(10, return_sequences=True))
])

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 10)          660       
                                                                 
 bidirectional (Bidirectiona  (None, None, 20)         1320      
 l)                                                              
                                                                 
Total params: 1,980
Trainable params: 1,980
Non-trainable params: 0
_________________________________________________________________


## Bean Search

Algoritmo utilizado en la generación de texto con RNN que mantiene un conjunto de posibles secuencias de salida en cada paso, eligiendo las k palabras más probables en cada paso. Se evalúa cada secuencia de salida utilizando una función de puntuación y se selecciona la secuencia de salida con la puntuación más alta. El Beam Search se utiliza para mejorar la calidad de la generación de texto y para evitar la selección de soluciones subóptimas.

In [ ]:
beam_width = 10
start_tokens = np.full([1000], 1)  # Utiliza el token 1 como inicio de la secuencia de salida
end_token = np.full([1000], 2)  # Utiliza el token 2 como fin de la secuencia de salida

decoder = tfa.seq2seq.beam_search_decoder.BeamSearchDecoder(
    cell=decoder_cell, beam_width=beam_width, output_layer=output_layer)

decoder_initial_state = tfa.seq2seq.beam_search_decoder.tile_batch(
    encoder_state, multiplier=beam_width)
outputs, _, _ = decoder(
    decoder_embeddings, start_tokens=start_tokens, end_token=end_token,
    initial_state=decoder_initial_state)
